# Documentacion

## Extraccion, transformacion y carga

1. Se crearon las bases de datos de **staging** y **data warehouse** en Microsoft sql server
    - **STG_WWBI**
    - **DWH_WWBI**
<img src="img_wbg/crear_base_datos.png" width=300 height=5>

2. Se crearon las tablas para la estructuracion de datos de ambas bases de datos

**Script para Staging**
    
<img src="img_wbg/tabla_staging_1.png" width=550 height=10>

<img src="img_wbg/tabla_staging_2.png" width=550 height=10>
    
**Script para Data Warehouse**
    
<img src="img_wbg/tabla_data_warehouse.png" width=550 height=10>

**Evidencia creacion de tablas para las bases de datos**

<img src="img_wbg/creacion_tablas.png" width=300 height=10>

3. En SPOON se realizaron las conexiones de las bases de datos que se encuentran en el programa Microsoft sql server anteriormente creadas, es decir, se crearon las conexiones para **STG_WWBI y DWH_WWBI**

<img src="img_wbg/conexiones_base_datos_spoon.png" width=400 height=10>

4. Posteriormente se creo una transformacion donde se extraen los ficheros CSV a una base de datos de staging 

<img src="img_wbg/staging_spoon.png" width=500 height=10>

Esta es la primera fase en los procesos ETL ya que es la preparatoria de los datos, donde se busca que los datos se encuentre en un mismo sistema llamado Staging.

En esta fase se crearon las salidas de datos, uno para el staging de país y salida para el staging de datos

- Preguntas
    - **¿Cuántas filas se han cargado en la tabla de staging País?:** Se han cargado 115 filas en STG_COUNTRY
    - **¿Cuántas filas se han cargado en la tabla de staging de datos?:** Se han cargado 10005 filas en STG_DATA
    - **¿Cuántas transformaciones has usado para realizar la carga?:** Para realizar la carga de los ficheros CSV a STAGING se uso una sola transformacion ya que este es un solo proceso, en donde se busca extraer los ficheros CSV a STAGING.
    - **¿Qué objetos has usado en estas transformaciones?:** Se han utilizado el obejto de entrada de un archivo csv y salida de tablas y ademas el componente de mapeo.
    - **¿Has usado el componente Start?:** Hasta este punto no se ha utilizado el componente **START** que se encuentra en *job* porque este componente se utiliza para la orquestacion de procesos ETL y posteriormente se crearan mas transformaciones para que se ejecuten en conjunto.
    
<img src="img_wbg/metricas_staging_spoon.png" width=550 height=10>

- Para este momento ya se analizaron los datos obtenidos y se validaron las caracteristicas correspondientes:
    - Se contrasto el numero de registros insertados
    - Se cargaron los datos que se usaran en las fases siguientes
    - Se elimino la duplicidad de datos
    - Se verifico el tipo de datos

5. Despues se realiza el proceso de transformacion y carga de datos desde STAGING a DATA WAREHOUSE de las dimensiones de country y metricas

<img src="img_wbg/dimension_spoon.png" width=550 height=10>

**¿Cómo se ha cargado la tabla “dim_metrica”? ¿Cuál es su origen?:** para la realizacion de la dimension de metrica, se tomo la informacion del fichero de datos CSV_DATA ya que este ya se encontraba modificado con la informacion de *indicator code e indicator name* que eran las columnas a utilizar para la dimension de metricas. 

Para cargar esta dimension primero se realizo una consulta SQL en la entrada de la tabla de DATA que me permitiera obtener solamente las columnas especificadas, como se muestra en la siguiente imagen. Para esta entrada de tabla no se utilizo el componente de mapeo porque los campos tienen los mismos nombres. 

<img src="img_wbg/dim_metrica_consulta_sql.png" width=600 height=10>

**Metricas de las tablas de dimension**

<img src="img_wbg/metricas_dimension_spoon.png" width=600 height=10>

6. Se realizo el proceso de tranformacion y carga desde STAGING a DATA WAREHOUSE de la tabla hechos

<img src="img_wbg/tabla_hechos_spoon.png" width=600 height=10>

- Primero se realizo una sentencia SQL en el area de staging_data que especificara que no reciben nulos desde el año 2000 hasta el 2016

<img src="img_wbg/tabla_hechos_sql_spoon.png" width=600 height=10>

- El segundo paso fue utilizar el componente de normalizar, para que me permitiera pivotar las columnas, ya que en el fichero de origen se encuentran todos los años en columnas y así es mas dificil utilizarlos para los trabajos posteriores, este componente nos permite pasar de tener los años en columnas a tener una sola columna de año y posterior se guardan los valores de la columna de años en una nueva columna, en este caso se llama IN_VALUE.

<img src="img_wbg/tabla_hechos_normalizacion_spoon.png" width=600 height=10>

- Preguntas:
    - **¿Qué componentes se han usado para crear la tabla de hechos?:**Los componentes que se han utilizado han sido la entrada de una tabla, el componente de transformar que se utilizo el de normalizacion de filas, el mapeo y la salida de tabla. 
    - **¿Cuántas filas se han cargado en la tabla de hechos?:**Se cargaron 16813
    - **Por qué se han multiplicado el número de filas de la tabla de hechos?:**Se han duplicado las filas ya que los valores que se encontraban en columnas pasaron hacer filas y por ende se reducieron las columnas, es decir, antes aparecia solamente un país en una fila despues de la normalizacion un país puede tener varias columnas que corresponden a los años que se pivotaron, lo que hizo que el volumen de las filas aumentaran. 
    
<img src="img_wbg/metrica_tabla_hechos_spoon.png" width=600 height=10>

7. Se crea un job

<img src="img_wbg/proceso_ETL_spoon.png" width=600 height=10>

- Esto permite ejecutar las transformaciones por orden 
- Se empieza con el componente START
- Despues se añaden las transformaciones, para este caso se colocaron 3 transformaciones para ejecutar que son:
    - STAGING_WWBI
    - DIM_WWBI
    - FACT_WWBI
- Preguntas:
    - **¿Se ha usado una transformación o una tarea?:** Para ejecutar las transformaciones anteriormente realizadas es necesario crear un job
    - **¿Por qué?:** El job permite orquestar y ejecutar estas transformaciones en el orden que se le especifique
    - **¿Qué tipo de objetos se han usado?:** Se han utilizado el objeto de START que con ese componente se empieza un proceso de ETL y tambien 3 transformaciones que fueron las que se realizaron.
    
<img src="img_wbg/metricas_proceso_ETL_spoon.png" width=600 height=10>

## Consultas SQL

- ¿Cuántos países pertenecen a cada grupo de ingresos (income group)?

<img src="img_wbg/income_group.png" width=600 height=10>

- ¿Cuántas métricas existen?

En la imagen se puede evidenciar tanto las metricas como la cantidad que existen y son 85 metricas 

<img src="img_wbg/metricas_existentes.png" width=600 height=10>

- ¿Cuantas metricas existen que tengan valor no nulo en el año 2000?

Teniendo en cuento los anteriores puntos se puede evidenciar que se añadio una sentencia SQL que me permitiera quitar todos los nulos desde el año 2000, antes de esta sentencia se cargaba un total de 10005 filas y despues se esta sentencia se cargaron 989 filas. Existen 85 metricas y en la otra columna la cantidad.

<img src="img_wbg/metrica_no_nulo.png" width=600 height=10>


## Cardinalidad

- Indicar la estructura del modelo de datos. Definir las tablas, sus relaciones y cardinalidades.

Para este caso se realizo una estructura de modelo en estrella, como se puede observar solo existe una tabla de hechos que se encuentra rodeada de dos tablas de dimensiones, una que es DIM_COUNTRY y otra DIM_METRICAS, este tipo de estructura de suele usar en datamarts ya que permite una explotacion de los datos de forma facil y ademas reduce el numero de cruces entre las tablas lo que permite que tenga un buen rendimiento. 

<img src="img_wbg/modelo_datos_power_BI.png" width=600 height=10>

# Visualizaciones

1. Evolución en el tiempo del “Empleo del sector público como parte del empleo remunerado” y el “Empleo del sector público como parte del empleo formal” para Argentina.

    - Para este informe se utilizo el **grafico de lineas** ya que este permite observar las metricas en dimension de tiempo y se busca mostrar una tendencia de estos datos.
    - Para obtener este grafico se utilizaron los campos de **ID_AÑO, DESC_METRICA, IN_VALUE y ID_PAIS**
    - Para el eje X se utilizo el **ID_AÑO**, para el eje Y **IN_VALUE**
    - En leyenda se utilizo **DESC_METRICA** y se filtro por "Public sector employment as a share of paid employment" y "Public sector employment as a share of formal employment"
    - **ID_PAIS** se filtro por ARG


<img src="img_wbg/visualizacion_argentina.png" width=600 height=10>


2. Evaluar la edad media de los empleados del sector privado y público por región.

    - Se realizo un **grafico de barras agrupadas** ya que permite comparar mejor diferentes categorias
    - Se utilizaron campos como **DESC_REGION, DESC_METRICA, IN_VALUE**
    - En leyenda se coloco en campo **DESC_METRICA** y se filtro por "Mean age of private paid employees", "Mean age of public paid employees"
    - El valor IN_VALUE se coloco como promedio
    
<img src="img_wbg/edad_media_empleados.png" width=600 height=10>

3. Realizar una gráfica del promedio del peso relativo de los cargos técnicos en los sectores privados y públicos a lo largo del tiempo.

    - Para este informe se utilizo un **grafico de areas apiladas** ya que permite representar el volumen e identificar cuanto aporta al total cada valor de la dimension
    - Los campos utilizados son **ID_AÑO, DESC_METRICA, IN_VALUE**
    - Se filtro el campo **DESC_METRICA** por "Relative wage of technicians in private sector (using clerk as reference)" y "Relative wage of technicians in public sector (using clerk as reference)"
    - En la leyenda se coloco el campo **DESC_METRICA**
    
<img src="img_wbg/cargos_tecnicos.png" width=600 height=10>

4. Obtener el promedio del peso por región del gasto en empleados públicos respecto al GDP y el gasto público.

    - Se utilizo el **grafico de barras 100% apiladas** ya que este tipo de graficos me permite ver los porcentajes que aportan cada subcategoria al total
    - Los campos que se utilizaron son **DESC_REGION, DESC_METRICA, IN_VALUE**
    - Se filtro el campo **DESC_METRICA** por "Wage bill as a percentage of GDP" y "Wage bill as a percentage of Public Expenditure"
     - En valores se utilizo **IN_VALUE**
     
<img src="img_wbg/gastos_empleados.png" width=600 height=10>